In [1]:
import pyspark
from pyspark.sql import SparkSession


In [2]:
# create the spark session
spark = SparkSession.builder\
    .master("spark://instance-20240222-020650.asia-southeast1-a.c.de-zoomcamp-412301.internal:7077")\
    .appName("test")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/26 06:30:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read in the green and yellow taxi data

In [3]:
# read the green taxi data for 2020 and 2021
df_green = spark.read.parquet('data/pq/green/*/*')


24/02/26 06:39:11 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/02/26 06:39:12 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:978)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [5]:
# rename the columns
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [4]:
# read the yellow taxi data for 2020 and 2021
df_yellow = spark.read.parquet('data/pq/yellow/*/*')


In [7]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



### Check the column for both dataset with same name

In [22]:
set(df_green.columns)& set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

The pickup and dropoff datetime is named differently for both dataset. Hence, we can rename the column accordingly

In [5]:
# rename the pickuptime and dropoff_time column
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime','pickup_datetime')\
    .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')

In [6]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime','pickup_datetime')\
    .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

In [25]:
# check again the common columns
set(df_green.columns) & set(df_yellow.columns) 

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

You may notice the order is not preserved. We want it to follow the original sequence

In [7]:
common_columns = []

for column in df_green.columns:
    if column in df_yellow.columns:
        common_columns.append(column)

In [8]:
common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

### Combine two dataset
If we want to combine this two dataset, we need to distinguish this them with `service_type`, hence we need to transform the column using the built-in function call `.lit()` in Spark

In [9]:
# import functions
from pyspark.sql import functions as F

In [10]:
df_green_sel = df_green \
            .select(common_columns)\
            .withColumn("service_type",F.lit('Green'))


In [11]:
df_yellow_sel = df_yellow\
            .select(common_columns)\
            .withColumn("service_type",F.lit('Yellow'))

In [12]:
df_yellow_sel.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- service_type: string (nullable = false)



In [13]:
## Union all the data
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [14]:
df_trips_data.groupBy("service_type").count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       Green| 2304517|
|      Yellow|39649199|
+------------+--------+



### Write SQL Query in Spark
Prior running the SQL Query, we need to register our data 

In [15]:
df_trips_data.createOrReplaceTempView("trips_data")

In [16]:
spark.sql("""
SELECT
    service_type,
    COUNT(1)
FROM 
    trips_data
GROUP BY
    service_type;
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       Green| 2304517|
|      Yellow|39649199|
+------------+--------+



In [17]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    revenue_zone, revenue_month, service_type
""")


In [18]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|         205|2020-

In [26]:
# in default, it have already saved in one parquet file only
df_result.write.parquet('data/report/revenue/')

# we can also specify the parition number via 
# df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')


### Read the report 

In [19]:
df_report = spark.read.parquet(
    'data/report/revenue/part-00000-15ce3da3-ba4f-4e9c-b9e9-b5c025caed0f-c000.snappy.parquet',
    header=True, inferSchema=True
)

In [20]:
df_report

DataFrame[revenue_zone: int, revenue_month: timestamp, service_type: string, revenue_monthly_fare: double, revenue_monthly_extra: double, revenue_monthly_mta_tax: double, revenue_monthly_tip_amount: double, revenue_monthly_tolls_amount: double, revenue_monthly_improvement_surcharge: double, revenue_monthly_total_amount: double, revenue_monthly_congestion_surcharge: double, avg_montly_passenger_count: double, avg_montly_trip_distance: double]

### Understand how the Spark Cluster work 

To understand how the group by work in spark cluster, we run a SQL query to explore more

### Understand the GROUP BY 

In [21]:
# register the df_green table 
df_green.createOrReplaceTempView('green')

In [30]:
df_green_revenue = spark.sql("""
SELECT
   
    date_trunc('hour', pickup_datetime) AS hour,
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
    
FROM
    green
WHERE
    pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
ORDER BY
    1,2

""")

In [31]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000002|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [32]:
# save the parquet file
df_green_revenue.write.parquet('data/report/revenue/green',mode='overwrite')

In [27]:
# try removing order by

df_green_revenue = spark.sql("""
SELECT
   
    date_trunc('hour', pickup_datetime) AS hour,
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
    
FROM
    green
WHERE
    pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
""")

In [29]:
# save the parquet file
df_green_revenue.write.parquet('data/report/revenue/green',mode='overwrite')

In [33]:
# register the df_yellow table 
df_yellow.createOrReplaceTempView('yellow')

In [34]:
df_yellow.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [35]:
df_yellow_revenue = spark.sql("""
SELECT
   
    date_trunc('hour', pickup_datetime) AS hour,
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
    
FROM
    yellow
WHERE
    pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
ORDER BY
    1,2

""")

In [38]:
# you repartition the file accordingly if you want
# In our case, we go with the default of 4 partitions
# df_yellow_revenue\
#     .repartition(20)\
#     .write.parquet('data/report/revenue/yellow', mode='overwrite')

df_yellow_revenue\
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

### Understand the JOIN 

Join two table with similar size

In [39]:
# rename the column to avoid conflict
df_green_revenue_temp = df_green_revenue\
            .withColumnRenamed('amount','green_amount')\
            .withColumnRenamed('number_records','green_number_records')

df_yellow_revenue_temp = df_yellow_revenue\
            .withColumnRenamed('amount','yellow_amount')\
            .withColumnRenamed('number_records','yellow_number_records')

In [40]:
df_green_revenue_temp

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint]

In [41]:
df_join = df_green_revenue_temp.join(
    df_yellow_revenue_temp,
    on=['hour','zone'],
    how='outer'
)

In [42]:
df_join.write.parquet('data/report/revenue/total')

In [44]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

Join one large and one small table

In [46]:
# large table
df_join = spark.read.parquet('data/report/revenue/total/')

# small table
df_zones = spark.read.parquet('zones/')

In [47]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [48]:
df_result

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint, LocationID: string, Borough: string, Zone: string, service_zone: string]

In [49]:
df_result.drop('LocationID','zone').write.parquet('tmp/revenue-zone')